This notebook is part of the [Machine Learning class](https://github.com/erachelson/MLclass) by [Emmanuel Rachelson](https://personnel.isae-supaero.fr/emmanuel-rachelson?lang=en).

License: CC-BY-SA-NC.

<div style="font-size:22pt; line-height:25pt; font-weight:bold; text-align:center;">Deep Learning and Convolutional Neural Networks</div>

This class supposes a basic knowledge of Artificial Neural Networks, Backpropagation and Stochastic Gradient Descent (as introduced in the previous class).

1. [Neural network training with Keras](#sec1)
2. [The Fashion-MNIST dataset](#sec2)
2. [Going deeper (first steps in Deep Learning)](#sec3)
3. [Avoiding overfitting](#sec4)
4. [Better optimizers](#sec5)
5. [CNN](#sec6)
6. [Going further](#sec7)

# 1. <a id="sec1"></a>Neural network training with [Keras](https://keras.io/)

The following parts are strongly inspired by this [tutorial](https://github.com/Matt3164/Fashion_Keras).

There are many good Neural Networks libraries on the net. Among the most reknowned are Tensorflow, Theano, Pytorch or Caffe.

Keras is a minimalist, modular neural network library that can use either Theano or TensorFlow as a backend. The primary motivation behind Keras is that you should be able to experiment fast and go from idea to result as quickly as possible. Thus, Keras is perfect for training purposes and tutorials for beginners. Building networks in Keras feels easy and natural. It includes some of the latest state-of-the-art algorithms for optimizers (Adam, RMSProp), normalization (BatchNorm), and activation layers (PReLU, ELU, LeakyReLU). Although, Keras does not support multi-GPU environments for training a network in parallel.

<img src="img/keras-logo-small.jpg" width="200px">

# 2. <a id="sec2"></a>The Fashion-MNIST dataset

**New data!**

Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes. Fashion-MNIST is a direct drop-in replacement for the original MNIST dataset for benchmarking machine learning algorithms. It shares the same image size and structure of training and testing splits but is more complex.

Here's an example of how the data looks like (each class takes three-rows):

<img src="img/fashion-mnist-sprite.png">

You can find the official benchmark [here](https://github.com/zalandoresearch/fashion-mnist) but it does not contain deep learning performances. The performance on DL methods is expected to be between 90% and 95% on the test set according to the [community](https://medium.com/tensorist/classifying-fashion-articles-using-tensorflow-fashion-mnist-f22e8a04728a).

**Let's load the data.**

One nice thing is that Fashion MNIST has become part of the standard computer vision benchmarks and is included with most libraries, including Keras.

In [ ]:
from keras.datasets import fashion_mnist

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

Alternatively, one can download the data directly. Uncomment and run the cell below if the one above didn't work for some reason.

In [ ]:
#!mkdir -p data/fashion
#!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
#!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz
#!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz
#!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz
#!mv *.gz data/fashion/

#from utils import mnist_reader
#X_train, y_train = mnist_reader.load_mnist('data/fashion', kind='train')
#X_test, y_test = mnist_reader.load_mnist('data/fashion', kind='t10k')

In [ ]:
print("Number of images for training: {}".format(X_train.shape[0]))
print("Image size: {}".format(X_train.shape[1:]))
print("Number of images for testing: {}".format(X_test.shape[0]))

So far, we have seen that neural networks take vectors as inputs, not images, so we need to "flatten" the 28x28 image into a 784-long single vector.

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 784)
X_test  = X_test.reshape(X_test.shape[0], 784);
print(X_train.shape)
print(X_test.shape)

Display the first image (now that we have flattened all images, we need to reshape them to 28x28 to display them as images).

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.imshow(X_train[0,:].reshape(28, 28), cmap='Greys')
plt.axis("off");

In [ ]:
gridSize=16

plt.figure(figsize=(10,10))
for ind in range(gridSize*gridSize):
    plt.subplot(gridSize, gridSize, ind+1)
    plt.imshow(X_train[ind, :].reshape(28, 28), cmap="Greys")
    plt.axis("off")

Here is the label map and the class count in the training and testing sets.

In [ ]:
import numpy as np

Labels = ["T-shirt/top",
"Trouser",
"Pullover",
"Dress",
"Coat",
"Sandal",
"Shirt",
"Sneaker",
"Bag",
"Ankle boot"]

l, c = np.unique(y_train, return_counts=True)
lt, ct = np.unique(y_test, return_counts=True)

for idx, count, count_test in zip(l, c, ct):
    print("{0}: {1} / {2}".format(Labels[idx], count, count_test))

**One-hot encoding** is the representation of the target class as a vector of 0s and 1s.

In [ ]:
from keras.utils import to_categorical
y_train_ohe = to_categorical(y_train)
y_test_ohe = to_categorical(y_test)
print(y_train_ohe.shape)

# <a id="sec3"></a> 3. Our first neural network in Keras

**Our first network in Keras**

Let's keep it simple. There is no hidden layer:
- input dimension is $28\times28=784$
- output dimension is 10 (10 classes)

So our network is simply:
$$ Y = \sigma(w^T x+b) $$

The softmax function $\sigma$ normalizes all outputs so that they sum to 1. For a $K$-dimensional vector $z$:
$$\sigma(z)_j = \frac{e^{z_j}}{\sum_{k=1}^K e^{z_k}}$$

We shall use the cross entropy loss function:

<img src="img/cross-entropy.png" width="700px">

Let's declare a [feed-forward neural network](https://keras.io/getting-started/sequential-model-guide/).

In [ ]:
from keras.models import Sequential
model = Sequential()

Let's now add the output layer, indicating that its input is a 784 vector corresponding to our 28x28 grayscale image and that the layer is a [fully connected](https://keras.io/layers/core/#dense) one.

In [ ]:
from keras.layers import Dense
model.add(Dense(units=10, input_dim=X_train.shape[1]))

Let's indicate that the [activation function](https://keras.io/activations/) for this output layer is a softmax.

In [ ]:
from keras.layers import Activation
model.add(Activation('softmax'))

We want to optimize the cross-entropy loss but we would also like to monitor the accuracy and the "top 3 categorical accuracy" [metrics](https://keras.io/metrics/) (whether the true class is among the three most probable predicted classes) along the training, so we define it as a function.

In [ ]:
import keras.metrics as metrics
def top3_acc(ytrue, ypred):
    return metrics.top_k_categorical_accuracy(ytrue, ypred, k=3)

Finally, we want to use stochastic gradient descent as an [optimizer](https://keras.io/optimizers/) and optimize the cross-entropy [loss](https://keras.io/losses/). All this allows to fully define our optimization task and to [*compile* our model](https://keras.io/models/model/).

In [ ]:
import keras.losses as losses
import keras.optimizers as optimizers

model.compile(loss=losses.categorical_crossentropy,
              optimizer=optimizers.SGD(lr=0.001),
              metrics=[metrics.categorical_accuracy, top3_acc])

**Let's train!**

In [ ]:
%%time
history = model.fit(X_train, y_train_ohe, validation_data=(X_test,y_test_ohe), epochs=400, batch_size=512)

It is common to plot different values obtained during training (by epoch). Keras provides the convenient [history](https://keras.io/visualization/#training-history-visualization) output to do so.

In [ ]:
plt.figure(figsize=(20,10))
plt.style.use('ggplot')
for i, (name, values) in enumerate(sorted(history.history.items())):
    plt.subplot(1, len(history.history.items()), i+1)
    plt.plot(values)
    plt.title(name)

In case you stopped the optimization early and did not run it for the 400 epochs, here is what you should get after those 400 epochs:

<img src="img/training_history_1.png">

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

predictions = model.predict(X_test).argmax(axis=1)

print("Accuracy: ")
print(accuracy_score(predictions, y_test))
print("\n\n")
print("Confusion matrix: ")
print(confusion_matrix(predictions, y_test))
print("\n\n")
print(classification_report(predictions, y_test, target_names=Labels))

Again, if you didn't run the optimization for 400 epochs, here is the result you should have found if you did:

```
Accuracy: 
0.5371



Confusion matrix: 
[[  0   0   0   0   0   0   0   0   0   0]
 [ 11 962  25  17   2   0   8   0   1   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [172  25  43 895  46   1  86   0   7   0]
 [ 18   7 379  35 847   1 145   0   4   0]
 [  0   0   1   1   1 982   0 997   7 998]
 [770   6 536  46  97   0 726   0  22   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [ 29   0  16   6   7  16  35   3 959   2]
 [  0   0   0   0   0   0   0   0   0   0]]



             precision    recall  f1-score   support

T-shirt/top       0.00      0.00      0.00         0
    Trouser       0.96      0.94      0.95      1026
   Pullover       0.00      0.00      0.00         0
      Dress       0.90      0.70      0.79      1275
       Coat       0.85      0.59      0.70      1436
     Sandal       0.98      0.33      0.49      2987
      Shirt       0.73      0.33      0.45      2203
    Sneaker       0.00      0.00      0.00         0
        Bag       0.96      0.89      0.93      1073
 Ankle boot       0.00      0.00      0.00         0

avg / total       0.89      0.54      0.64     10000```

Let's identify some misclassified items.

In [ ]:
import random

wrong_predictions = (predictions != y_test)
n_wp = wrong_predictions.sum()

N = 4
for i in range(N):
    idx = random.choice(range(n_wp))
    plt.figure()
    plt.imshow(X_test[wrong_predictions,::][idx,::].reshape(28, 28), cmap="Greys")
    plt.axis("off")
    plt.title("Predicted: {0} // Truth: {1}".format(
        Labels[predictions[wrong_predictions][idx]],
        Labels[y_test[wrong_predictions][idx]]
    ))

# 4. <a id="sec4"></a>Going deeper (first steps in Deep Learning)

Deep Learning refers to learning artificial neural networks that have many layers. This brings a whole set of additional difficulties.

**Vanishing gradients**

Recall the shape of the sigmoid activation function?

In [ ]:
def sigmoid(x):
    return 1./(1. + np.exp(-x))

XX = np.arange(-5,5.,0.1)
plt.plot(XX,sigmoid(XX));

Let's plot the gradient of this function.

In [ ]:
def sigmoid_der(x):
    y = sigmoid(x)
    return y*(1.-y)

XX = np.arange(-5,5.,0.1)
plt.plot(XX,sigmoid(XX));
plt.plot(XX,sigmoid_der(XX));

Do you remember that during backpropagation, the $\delta_j$ were recursively obtained by:
$$\delta_j = \sigma'(y_j) \sum_{l\in L_j} \delta_l w_{jl}$$

This poses a major problem when the networks become deeper: at each layer, we multiply our gradients by $\sigma'(y_j)$ which is much smaller than 1. So the gradient we want to back-propagate shrinks to zero and all our weight updates become zero.

This is called the **vanishing gradient** problem.

To avoid this problem, we introduce a new type of activation function: the Rectified Linear Unit (ReLU).
$$\sigma(y) = \max\{0,y\}$$

The key property of this function is that its derivative is either zero or one.

In [ ]:
def relu(x):
    return np.maximum(0.,x)

plt.plot(XX,relu(XX));

**Caveat of using ReLU activation functions**

Although they allow us to train deep networks, ReLU functions have their downsides.
- Unbounded values: the output of a layer is not bounded anymore, causing possible divergence.
- Dying ReLU neurons: the backpropagation of gradients can push the input weigths towards values such that $\sigma(y)=0$ all the time. Then all future backpropagations will leave these weights unchanged: the neuron is dead.

Some advanced methods can compensate these weaknesses but are out of the scope of this introduction.

Let's build a 5-layered network with ReLU neurons.

In [ ]:
X_train.shape[1]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
import keras.metrics as metrics
import keras.losses as losses
import keras.optimizers as optimizers


def top3_acc(ytrue, ypred):
    return metrics.top_k_categorical_accuracy(ytrue, ypred, k=3)

model = Sequential()
model.add(Dense(units=200, input_dim=X_train.shape[1]))
model.add(Activation('relu'))
model.add(Dense(units=150))
model.add(Activation('relu'))
model.add(Dense(units=50))
model.add(Activation('relu'))
#model.add(Dense(units=30))
#model.add(Activation('relu'))
model.add(Dense(units=10))
model.add(Activation('softmax'))

# Change decay for better results
# lr: 1e-3, decay: 0

N = X_train.shape[0]
batch_size = 32
n_batch = int(N/batch_size)

model.compile(loss=losses.categorical_crossentropy,
              optimizer=optimizers.SGD(),
              metrics=[metrics.categorical_accuracy, top3_acc])

In [ ]:
%%time
history = model.fit(X_train, y_train_ohe, validation_data=(X_test, y_test_ohe), epochs=50, batch_size=512)

In [ ]:
plt.figure(figsize=(20,10))
plt.style.use('ggplot')
for i, (name, values) in enumerate(sorted(history.history.items())):
    plt.subplot(1, len(history.history.items()), i+1)
    plt.plot(values)
    plt.title(name)

In case you stopped the optimization early and did not run it for the 150 epochs, here is what you should get after those 150 epochs:

<img src="img/training_history_2_overfit.png">

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

predictions = model.predict(X_test).argmax(axis=1)

print("Accuracy: ")
print(accuracy_score(predictions, y_test))
print("\n\n")
print("Confusion matrix: ")
print(confusion_matrix(predictions, y_test))
print("\n\n")
print(classification_report(predictions, y_test, target_names=Labels))

Again, if you didn't run the optimization for 150 epochs, here is the result you should have found if you did:

```
TODO
```

In [ ]:
import random

wrong_predictions = (predictions != y_test)
n_wp = wrong_predictions.sum()

N = 4
for i in range(N):
    idx = random.choice(range(n_wp))
    plt.figure()
    plt.imshow(X_test[wrong_predictions,::][idx,::].reshape(28, 28), cmap="Greys")
    plt.axis("off")
    plt.title("Predicted: {0} // Truth: {1}".format(
        Labels[predictions[wrong_predictions][idx]],
        Labels[y_test[wrong_predictions][idx]]
    ))

# 5. <a id="sec5"></a>Avoiding overfitting 

If you look closely, our network still suffers from overfitting. 

We are going to introduce four techniques to avoid that:

**Early-stopping**

TODO

**Dropout**

The idea of dropout is very simple. At each training epoch, some neurons are either "dropped out" of the network with probability $1 − p$ or kept with probability $p$, so that training occurs only on a pruned network. Incoming and outgoing edges to a dropped-out neuron are also removed. Only the pruned network is trained on the data in that epoch. The removed neurons are then reinserted into the network with their original weights.<br>
[**Dropout: A Simple Way to Prevent Neural Networks from Overfitting**. Srivastava et al., Journal of Machine Learning Research (2014).](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf)

**Regularization**

Regularization is an idea borrowed to the general field of statistical learning. In the bias-variance compromise, a good way of allowing low bias while keeping low variance is to penalize the objective function with the norm of the parameters' vector. For example, we could include $\|\theta\|_1$ in the loss function.

**Data augmentation**

Data augmentation consists in artificially creating data by applying transformations on the input data. This way, we provide more examples to the classifier, that avoids overfitting by precisely overfitting the perturbed artificial data. It is particularly efficient when working with image data.

In this section we only experiment with dropout.

In [ ]:
from keras.layers import Dropout

dropout_rate=0.1

model = Sequential()
model.add(Dense(units=200, input_dim=X_train.shape[1]))
model.add(Activation('relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(units=150))
model.add(Activation('relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(units=50))
model.add(Activation('relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(units=30))
model.add(Activation('relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(units=10))
model.add(Activation('softmax'))

model.compile(loss=losses.categorical_crossentropy,
              optimizer=optimizers.SGD(lr=0.001, decay=0., nesterov=False),
              metrics=[metrics.categorical_accuracy, top3_acc])

In [ ]:
%%time
history = model.fit(X_train, y_train_ohe, validation_data=(X_test,y_test_ohe), epochs=150, batch_size=64)

In [ ]:
plt.figure(figsize=(20,10))

for i, (name, values) in enumerate(sorted(history.history.items())):
    plt.subplot(1, len(history.history.items()), i+1)
    plt.plot(values)
    plt.title(name)

In case you stopped the optimization early and did not run it for the 150 epochs, here is what you should get after those 150 epochs:

<img src="img/training_history_3.png">

In [ ]:
predictions = model.predict(X_test).argmax(axis=1)

print("Accuracy: ")
print(accuracy_score(predictions, y_test))
print("\n\n")
print("Confusion matrix: ")
print(confusion_matrix(predictions, y_test))
print("\n\n")
print(classification_report(predictions, y_test, target_names=Labels))

Again, if you didn't run the optimization for 150 epochs, here is the result you should have found if you did:

```
Accuracy: 
0.8829



Confusion matrix: 
[[825   2  14  17   0   0 118   0   5   0]
 [  1 963   0   5   0   0   1   0   0   0]
 [ 14   1 785   8  85   0 107   0   4   0]
 [ 28  25  12 894  24   1  26   0   3   0]
 [  3   4 123  44 851   0  70   0   3   0]
 [  1   0   0   2   0 952   0  13   3   5]
 [120   3  64  28  39   0 664   0  11   1]
 [  0   0   0   1   0  31   0 968   6  32]
 [  8   2   2   1   1   2  14   0 965   0]
 [  0   0   0   0   0  14   0  19   0 962]]



             precision    recall  f1-score   support

T-shirt/top       0.82      0.84      0.83       981
    Trouser       0.96      0.99      0.98       970
   Pullover       0.79      0.78      0.78      1004
      Dress       0.89      0.88      0.89      1013
       Coat       0.85      0.78      0.81      1098
     Sandal       0.95      0.98      0.96       976
      Shirt       0.66      0.71      0.69       930
    Sneaker       0.97      0.93      0.95      1038
        Bag       0.96      0.97      0.97       995
 Ankle boot       0.96      0.97      0.96       995

avg / total       0.88      0.88      0.88     10000
```

# 6. <a id="sec6"></a>Better optimizers

Stochastic Gradient Descent is our core method but improved optimizers are readily implemented in Keras. Most of them, like ADAM or RMSprop rely on the idea of including momentum in the gradient descent.

# 7. <a id="sec7"></a>Convolutional Neural Networks

The main factor which makes deep learning so useful for computer vision is the use of convolutions. Convolutional networks exploit the fact that the data is actually an image in the learning while decreasing the number of weights in the network. To do this, they define **convolution filters** that brush across the image. Such a filter defines a so-called **feature map** that shares the weights of the filter. The result of applying a feature map on an image is a new image of lower resolution, where each pixel is the result of the convolution of the filter with a set of pixels from the input image, as illustrated on the figure below.

<img src="img/convnet.gif" width="700px">

Another specificity of convolutional networks is **pooling**. Pooling is the operation of down-sampling the image by grouping together certain pixels. The most common pooling operation takes the maximum value over a certain window.

The following network is quite simple. It is a ConvNet, a family of networks built by stacking Conv/Pool blocks upon each other and adding a final set of dense layers as in the network below:
- Input layer: images of size $28\times 28$ with a single channel
- Convolutional layer of 16 feature maps with $5\times 5$ filters and ReLU activation functions
- Max pooling layer by blocks of size $2 \times 2$
- Convolutional layer of 16 feature maps with $5\times 5$ filters and ReLU activation functions
- Max pooling layer by blocks of size $2 \times 2$
- Fully connected layer with 200 ReLU neurons
- Fully connected output layer with 10 softwax neurons


In [ ]:
from keras.layers import Conv2D, MaxPool2D, MaxPooling2D, Flatten

model = Sequential()

model.add(Conv2D(16, (5, 5), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(16, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(200, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(10, activation='softmax'))

model.compile(loss=losses.categorical_crossentropy,
              optimizer=optimizers.SGD(lr=0.001, decay=0., nesterov=False),
              metrics=[metrics.categorical_accuracy, top3_acc])

Let's save a picture of our model.

In [ ]:
# requires pydot and graphviz (skip this step if you don't want to install them)
from keras.utils import plot_model
plot_model(model, to_file='model.png')

Let's train.

In [ ]:
%%time
history = model.fit(X_train.reshape(-1, 28, 28, 1), y_train_ohe, validation_data=(X_test.reshape(-1, 28, 28, 1),y_test_ohe), epochs=150, batch_size=64)

In [ ]:
plt.figure(figsize=(20,10))

for i, (name, values) in enumerate(sorted(history.history.items())):
    plt.subplot(1, len(history.history.items()), i+1)
    plt.plot(values)
    plt.title(name)

In case you stopped the optimization early and did not run it for the 150 epochs, here is what you should get after those 150 epochs:

<img src="img/training_history_4.png">

In [ ]:
predictions = model.predict(X_test.reshape(-1, 28, 28, 1)).argmax(axis=1)

print("Accuracy: ")
print(accuracy_score(predictions, y_test))
print("\n\n")
print("Confusion matrix: ")
print(confusion_matrix(predictions, y_test))
print("\n\n")
print(classification_report(predictions, y_test, target_names=Labels))

Again, if you didn't run the optimization for 150 epochs, here is the result you should have found if you did:

```
Accuracy: 
0.8825



Confusion matrix: 
[[784   3  13  12   2   0  90   0   1   0]
 [  1 970   1   7   1   0   5   0   0   0]
 [ 11   0 755   8  45   0  58   0   0   1]
 [ 26  17   7 897  28   0  32   0   5   1]
 [  2   4  95  33 823   0  50   0   3   0]
 [  5   0   1   0   0 955   0  13   1   5]
 [163   6 125  37  97   1 752   0  14   0]
 [  0   0   1   0   0  32   0 972   7  44]
 [  8   0   2   6   4   2  13   0 968   0]
 [  0   0   0   0   0  10   0  15   1 949]]



             precision    recall  f1-score   support

T-shirt/top       0.78      0.87      0.82       905
    Trouser       0.97      0.98      0.98       985
   Pullover       0.76      0.86      0.80       878
      Dress       0.90      0.89      0.89      1013
       Coat       0.82      0.81      0.82      1010
     Sandal       0.95      0.97      0.96       980
      Shirt       0.75      0.63      0.69      1195
    Sneaker       0.97      0.92      0.95      1056
        Bag       0.97      0.97      0.97      1003
 Ankle boot       0.95      0.97      0.96       975

avg / total       0.88      0.88      0.88     10000
```

# <a id="sec8"></a> 8. Going further

Deep Learning is a hot topic with new contributions almost every week. Interested readers can investigate further the topics above to get a deeper understanding or explore the following keywords for example:
- batch normalization
- recurrent networks and LSTM
- pretrain / fine-tune
- pre-trained networks
- generative adversarial networks